<a href="https://colab.research.google.com/github/MFahadHussain/MachineLearning/blob/main/NCAI_Model_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#✅ Reusable Model Optimization Notebook Template (PyTorch)


In [1]:

# 📌 Install Required Libraries
!pip install torch torchvision onnx --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 63.1 MB/s eta 0:00:00


In [1]:
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import torch.quantization as quant
import torchvision
import torchvision.transforms as transforms
import time
import os

In [2]:
# ✅ CONFIGURATION SECTION
MODEL_TYPE = "classification"  # or 'detection'
DATASET = "cifar10"  # or path to your custom dataset
PRUNE_PERCENTAGE = 0.3  # 30% pruning
APPLY_QUANTIZATION = True
EPOCHS_AFTER_PRUNING = 5


In [3]:
# ✅ DATA LOADING BLOCK
if DATASET == "cifar10":
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)


100%|██████████| 170M/170M [00:02<00:00, 79.9MB/s]


In [4]:
# ✅ MODEL LOADING BLOCK (EXAMPLE: SimpleCNN)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)

In [5]:
# ✅ PRUNING BLOCK
parameters_to_prune = (
    (model.conv1, 'weight'),
    (model.conv2, 'weight'),
    (model.fc1, 'weight')
)
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=PRUNE_PERCENTAGE
)
print(f"Applied {PRUNE_PERCENTAGE*100}% pruning")

Applied 30.0% pruning


In [6]:
# ✅ FINE-TUNING AFTER PRUNING
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(EPOCHS_AFTER_PRUNING):
    model.train()
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
print("Fine-tuning completed after pruning")

Fine-tuning completed after pruning


In [7]:
# ✅ QUANTIZATION BLOCK
if APPLY_QUANTIZATION:
    model.cpu()
    model.qconfig = quant.get_default_qconfig('fbgemm')
    quant.prepare(model, inplace=True)
    with torch.no_grad():
        for inputs, labels in trainloader:
            model(inputs)
    quant.convert(model, inplace=True)
    print("Quantization applied")

/usr/local/lib/python3.11/dist-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Quantization applied


In [8]:
# ✅ EVALUATION BLOCK
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for inputs, labels in testloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test Accuracy After Optimization: {100 * correct / total:.2f}%")

AttributeError: 'Conv2d' object has no attribute 'weight_mask'

In [9]:
# ✅ SAVE MODEL
os.makedirs("/content/optimized_model", exist_ok=True)
torch.save(model.state_dict(), "/content/optimized_model/optimized_model.pth")
print("Optimized model saved to /content/optimized_model/optimized_model.pth")

# ✅ READY TO USE FOR ANY MODEL
# Just replace SimpleCNN with your custom model,
# adjust dataset loader, and tune PRUNE_PERCENTAGE and EPOCHS_AFTER_PRUNING.

Optimized model saved to /content/optimized_model/optimized_model.pth
